In [2]:
from google.oauth2 import service_account

# Replace with the path to your service account key file
filename = "key.json"
credentials = service_account.Credentials.from_service_account_file(filename)


In [3]:
from llama_index.llms.vertex import Vertex

# Initialize the Vertex AI model
llm = Vertex(
    model="gemini-pro",  # Specify the model, e.g., "text-bison" or "gemini-pro"
    project=credentials.project_id,
    credentials=credentials,
    temperature=0.0,  # Adjust as needed
    additional_kwargs={}
)


In [4]:
response = llm.complete("Hello, this is a sample text.")
print(response.text)


Hello! I am happy to help you with your sample text. What would you like me to do with it? I can summarize it, translate it, answer questions about it, or even write a creative piece based on it. Just let me know what you need.


In [14]:
# Define your custom function
def get_weather(city):
    return f"The weather in {city} is sunny with a temperature of 25°C."

# Function registry
function_registry = {
    "get_weather": get_weather,
}

# Function to call the appropriate function based on LLM output
def call_function(func_name, args):
    if func_name in function_registry:
        return function_registry[func_name](*args)
    else:
        return f"Function '{func_name}' not found!"

# Example usage
query = "What is the weather in New York?"
if "get_weather" in query.lower():
    city = "New York"  # Extract dynamically in real scenarios
    function_response = call_function("get_weather", [city])
    print("Function Response:", function_response)
else:
    response = llm.complete(query)
    print("LLM Response:", response.text)


LLM Response: I do not have access to real-time information, including weather updates. To get the most accurate and up-to-date weather information for New York, I recommend checking a reliable weather source such as the National Weather Service or a local news station. 

Would you like me to try and find some weather information for you from a source that is not real-time? 



In [29]:
from llama_index.core.llms import ChatMessage, MessageRole

# Initialize the chat model
chat_llm = Vertex(model="gemini-pro", project=credentials.project_id, credentials=credentials)

# Define the conversation
messages = [
    ChatMessage(role=MessageRole.SYSTEM, content="Reply in French."),
    ChatMessage(role=MessageRole.USER, content="Hello"),
]

# Generate a response
chat_response = chat_llm.chat(messages=messages)
print(chat_response.message.content)


Bonjour ! 👋 

Comment allez-vous aujourd'hui ? 😊


In [ ]:
# Example function to retrieve weather data
def get_weather(city):
    return f"The weather in {city} is sunny with a temperature of 25°C."

# Example function to retrieve stock prices
def get_stock_price(symbol):
    return f"The current stock price of {symbol} is $150."

# Registry to map function names to callable functions
function_registry = {
    "get_weather": get_weather,
    "get_stock_price": get_stock_price,
}

# Function to call appropriate tool
def call_function(func_name, args):
    if func_name in function_registry:
        return function_registry[func_name](*args)
    else:
        return f"Function '{func_name}' not found!"


In [ ]:
from llama_index.core.llms import ChatMessage, MessageRole

# Define a chat session
def chat_with_tools(messages):
    response = chat_model.chat(messages)
    return response.message["content"], response.message.get("function_call")

# Example chat session
messages = [
    ChatMessage(role=MessageRole.SYSTEM, content="You can retrieve weather or stock prices."),
    ChatMessage(role=MessageRole.USER, content="What is the weather in New York?")
]

response, function_call = chat_with_tools(messages)

# Handle function calls
if function_call:
    func_name = function_call["name"]
    func_args = eval(function_call["arguments"])  # Safely parse arguments
    function_response = call_function(func_name, func_args)
    print(f"Function Response: {function_response}")
else:
    print(f"Chat Response: {response}")


In [ ]:
def process_query(query):
    messages = [
        ChatMessage(role=MessageRole.SYSTEM, content="You can call functions for data retrieval."),
        ChatMessage(role=MessageRole.USER, content=query)
    ]

    response, function_call = chat_with_tools(messages)

    if function_call:
        func_name = function_call["name"]
        func_args = eval(function_call["arguments"])
        return call_function(func_name, func_args)
    return response

# Test queries
print(process_query("Get the weather in San Francisco."))
print(process_query("Tell me the stock price of AAPL."))


In [5]:
import requests
from dotenv import load_dotenv
load_dotenv()
import os
api_key = os.getenv("GeoCoordinates_api_key")

In [6]:
import requests
from dotenv import load_dotenv
load_dotenv()
import os
api_key = os.getenv("GeoCoordinates_api_key")

def get_coordinates(api_key, location):
    """
    Get latitude and longitude of a location using Google Maps Geocoding API.

    Args:
        api_key (str): Your Google Maps API key.
        location (str): The location name or address.

    Returns:
        dict: A dictionary with 'latitude' and 'longitude'.
    """
    url = f"https://maps.googleapis.com/maps/api/geocode/json"
    params = {
        "address": location,
        "key": api_key
    }
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK':
            result = data['results'][0]
            coordinates = result['geometry']['location']
            return {
                "latitude": coordinates['lat'],
                "longitude": coordinates['lng']
            }
        else:
            raise Exception(f"Error from API: {data['status']}")
    else:
        raise Exception(f"HTTP Error: {response.status_code}")



import requests

def get_location_from_coordinates(api_key, latitude, longitude):
    """
    Get location (city, state, country) from latitude and longitude using Google Maps API.

    Args:
        api_key (str): Your Google Maps API key.
        latitude (float): Latitude of the location.
        longitude (float): Longitude of the location.

    Returns:
        str: A formatted address or city name.
    """
    url = f"https://maps.googleapis.com/maps/api/geocode/json"
    params = {
        "latlng": f"{latitude},{longitude}",
        "key": api_key
    }
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK':
            # Extract the formatted address
            result = data['results'][0]
            address = result['formatted_address']
            return address
        else:
            raise Exception(f"Error from API: {data['status']}")
    else:
        raise Exception(f"HTTP Error: {response.status_code}")

In [7]:
import requests

# Function to get coordinates from a location
def get_coordinates(api_key, location):
    """
    Get latitude and longitude of a location using Google Maps Geocoding API.

    Args:
        api_key (str): Your Google Maps API key.
        location (str): The location name or address.

    Returns:
        dict: A dictionary with 'latitude' and 'longitude'.
    """
    url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {"address": location, "key": api_key}
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK':
            result = data['results'][0]
            coordinates = result['geometry']['location']
            return {"latitude": coordinates['lat'], "longitude": coordinates['lng']}
        else:
            return {"error": f"Error from API: {data['status']}"}
    else:
        return {"error": f"HTTP Error: {response.status_code}"}


# Function to get location from coordinates
def get_location_from_coordinates(api_key, latitude, longitude):
    """
    Get location (city, state, country) from latitude and longitude using Google Maps API.

    Args:
        api_key (str): Your Google Maps API key.
        latitude (float): Latitude of the location.
        longitude (float): Longitude of the location.

    Returns:
        str: A formatted address or city name.
    """
    url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {"latlng": f"{latitude},{longitude}", "key": api_key}
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK':
            result = data['results'][0]
            return result['formatted_address']
        else:
            return {"error": f"Error from API: {data['status']}"}
    else:
        return {"error": f"HTTP Error: {response.status_code}"}

# Function registry
function_registry = {
    "get_coordinates": get_coordinates,
    "get_location_from_coordinates": get_location_from_coordinates,
}

# Function caller
def call_function(func_name, args):
    if func_name in function_registry:
        return function_registry[func_name](*args)
    else:
        return {"error": f"Function '{func_name}' not found!"}



In [9]:
# try the fucntions
print(call_function("get_coordinates", [api_key, "New York"]))


{'error': 'Error from API: REQUEST_DENIED'}


In [30]:
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.llms.vertex import Vertex

# Initialize Vertex AI chat model
def initialize_chat_model(credentials):
    return Vertex(
        model="gemini-pro",
        project=credentials.project_id,
        credentials=credentials,
        temperature=0.7
    )

# Chat with function integration
def chat_with_tools(chat_model, api_key, user_query):
    messages = [
        ChatMessage(role=MessageRole.SYSTEM, content="You can retrieve geographic information."),
        ChatMessage(role=MessageRole.USER, content=user_query)
    ]

    response = chat_model.chat(messages)
    content = response.message["content"]
    function_call = response.message.get("function_call")

    if function_call:
        func_name = function_call["name"]
        args = eval(function_call["arguments"])  # Parse arguments safely
        args.append(api_key)  # Append the API key as the last argument
        return call_function(func_name, args)
    else:
        return {"response": content}


In [ ]:
def chat_with_tools(chat_model, api_key, user_query):
    messages = [
        ChatMessage(role=MessageRole.SYSTEM, content="You can retrieve geographic information."),
        ChatMessage(role=MessageRole.USER, content=user_query)
    ]

    # Send messages to chat model
    response = chat_model.chat(messages)

    return response

   

In [52]:

chat_model = initialize_chat_model(credentials)

In [ ]:
# Test 1: Get coordinates
query1 = "What are the coordinates of New York?"
result1= chat_with_tools(chat_model, api_key, query1)


ChatResponse(message=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': []}, blocks=[TextBlock(block_type='text', text='The coordinates of New York City are 40.7128° N, 74.0060° W.')]), raw={'_raw_response': candidates {
  content {
    role: "model"
    parts {
      text: "The coordinates of New York City are 40.7128° N, 74.0060° W."
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.0402832031
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0942382812
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.166015625
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0549316406
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.099609375
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.06738281

In [ ]:
result1.con

In [44]:
# Test 2: Get location from coordinates
query2 = "What location corresponds to latitude 40.7128 and longitude -74.0060?"
result2 = chat_with_tools(chat_model, api_key, query2)
print("Result 2:", result2)

Debug Response: assistant: The coordinates you provided, latitude 40.7128 and longitude -74.0060, correspond to **New York City, New York, USA**. More specifically, they point to a location in Lower Manhattan, near the intersection of Broadway and Wall Street. This area is known for its financial institutions, historical landmarks, and bustling atmosphere. 

Here are some of the notable landmarks located near these coordinates:

* **The World Trade Center**: This iconic skyscraper complex includes the One World Trade Center, the tallest building in the Western Hemisphere. 
* **The 9/11 Memorial & Museum**: This poignant site commemorates the victims of the September 11th terrorist attacks.
* **The Charging Bull**: This bronze sculpture is a popular tourist destination and symbol of Wall Street's financial power.
* **Battery Park**: This waterfront park offers stunning views of the Statue of Liberty and the New York Harbor.
* **The New York Stock Exchange**: This historic building is th

# tesing firbase functions 

In [17]:
from pydantic import BaseModel
from typing import Optional, Dict, List
import csv
import firebase_admin
from firebase_admin import credentials, firestore
from datetime import datetime
# from geopy.distance import geodesic

import io

In [ ]:
# Initialize Firebase
cred = credentials.Certificate('key_firebase.json')
firebase_admin.initialize_app(cred)
db = firestore.client()

In [19]:
# Helper functions
def serialize_firestore_document(doc):
    data = doc.to_dict()
    data['id']=doc.id
    for key, value in data.items():
        if isinstance(value, firestore.GeoPoint):
            data[key] = {
                "latitude": value.latitude,
                "longitude": value.longitude
            }
    return data

def is_within_radius(center, point, radius_km):
    center_coords = (center["latitude"], center["longitude"])
    point_coords = (point["latitude"], point["longitude"])
    return geodesic(center_coords, point_coords).km <= radius_km

# Pydantic schema
class ProductSchema(BaseModel):
    name: str
    description: str
    price: float
    quantity: int
    location: Optional[Dict[str, float]]


In [20]:
def get_all_products():
    try:
        products = db.collection('products').stream()
        product_list = [serialize_firestore_document(doc) for doc in products]
        return product_list
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


In [33]:
api_key = os.getenv("GeoCoordinates_api_key")

In [34]:
import requests

# Function to get coordinates from a location
def get_coordinates(api_key, location):
    """
    Get latitude and longitude of a location using Google Maps Geocoding API.

    Args:
        api_key (str): Your Google Maps API key.
        location (str): The location name or address.

    Returns:
        dict: A dictionary with 'latitude' and 'longitude'.
    """
    url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {"address": location, "key": api_key}
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK':
            result = data['results'][0]
            coordinates = result['geometry']['location']
            return {"latitude": coordinates['lat'], "longitude": coordinates['lng']}
        else:
            return {"error": f"Error from API: {data['status']}"}
    else:
        return {"error": f"HTTP Error: {response.status_code}"}


# Function to get location from coordinates
def get_location_from_coordinates(api_key, latitude, longitude):
    """
    Get location (city, state, country) from latitude and longitude using Google Maps API.

    Args:
        api_key (str): Your Google Maps API key.
        latitude (float): Latitude of the location.
        longitude (float): Longitude of the location.

    Returns:
        str: A formatted address or city name.
    """
    url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {"latlng": f"{latitude},{longitude}", "key": api_key}
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK':
            result = data['results'][0]
            return result['formatted_address']
        else:
            return {"error": f"Error from API: {data['status']}"}
    else:
        return {"error": f"HTTP Error: {response.status_code}"}

# Function registry
function_registry = {
    "get_coordinates": get_coordinates,
    "get_location_from_coordinates": get_location_from_coordinates,
}

# Function caller
def call_function(func_name, args):
    if func_name in function_registry:
        return function_registry[func_name](*args)
    else:
        return {"error": f"Function '{func_name}' not found!"}

